## Instances and extensions

This chapter will cover some of the different quantum variational algorithms:

* [Variational Quantum Eigensolver (VQE)]((https://arxiv.org/abs/1304.3061))
* [Variational Quantum Deflation (VQD)]((https://arxiv.org/abs/1805.08138))
* [Subspace Search VQE (SSVQE)](https://arxiv.org/abs/1810.09434)
* [Quantum Sampling Regression (QSR)](https://arxiv.org/pdf/2012.02338)

More advanced users may want to combine some of the ideas in these routines in order to come up with a custom algorithm fitting their needs best. This is beyond the scope of what we can explicitly cover in the course, nonetheless we encourage students to explore this idea and share their findings with the community!

## Variational Quantum Eigensolver (VQE)

[VQE](https://arxiv.org/abs/1304.3061) is one of the most widely used variational quantum algorithms, and arguably the first one to appear; setting up a template for other algorithms to build upon. Its layout is simple, we label iterations within the optimization loop with the index $i$, starting at $i=0$:

* If it's the initial iteration (i.e. $i=0$), bootstrap the optimization loop. Each optimizer will be bootstrapped in a different way, leading to an initial set of parameter vectors $\Theta_0 := \{\vec\theta_{0,j} \; | \; j \in \mathcal{J}_\text{opt}^0\}$ (e.g. from an initial point $\vec\theta_0$).
* Apply the ansatz:
    * Go from the state $|0\rangle$ to the reference state $|\rho\rangle$.
    * For all parameter vectors in the set $\Theta_i := \{\vec\theta_{i,j} \; | \; j \in \mathcal{J}_\text{opt}^i\}$ determined by the optimizer, apply the variational form $U_V(\vec\theta_{i,j})$; leading to the state(s) $|\psi(\vec\theta_{i,j})\rangle = U_V(\vec\theta_{i,j})|\rho\rangle$.
* Evaluate the cost function $C(\vec\theta_{i,j})$ for all prepared states by calling the quantum oracle (e.g. an IBM Quantum backend via the Runtime architecture).
* Use a [classical optimizer](optimization.ipynb) to choose the next set of parameters $\Theta_{i+1}$.
* Repeat until convergence is reached.

As previously explained, this is really just a classical optimization loop in which we call a quantum oracle in order to evaluate the cost function. The reason why several states may need to be evaluated on each iteration is that the optimizer could require more than just one evaluation to determine the next iteration or to assess convergence. For instance, gradient based optimizers may need additional evaluations in order to approximate said gradient numerically.

## Variational Quantum Deflation (VQD)

[VQD](https://arxiv.org/abs/1805.08138) is an iterative method that extends VQE to obtain the $k$ first eigenvalues of an observable $\hat{H}$ with eigenvalues $\{\lambda_0, \lambda_1,...,\lambda_{N-1}\}$, with $N\geq k$, instead of only the first. For the rest of this section we will assume without loss of generality that $\lambda_0\leq\lambda_1\leq...\leq\lambda_{N-1}$.


The general idea behind VQD is that, first, you use VQE as usual to get the lowest eigenvalue $\lambda_0 := C_0(\vec\theta^0) \equiv C_\text{VQE}(\vec\theta^0)$ with the corresponding (approximate) eigenstate $|\psi(\vec{\theta^0})\rangle$ for some optimal parameter vector $\vec{\theta^0}$. Then, in order to obtain the next eigenvalue $\lambda_1>\lambda_0$, instead of minimizing the cost function $C_0(\vec{\theta}) := \langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle$, we optimize

$$
C_1(\vec{\theta}) := 
C_0(\vec{\theta})+ \beta_0 |\langle \psi(\vec{\theta})| \psi(\vec{\theta^0})\rangle  |^2 
% = 
% \langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle + 
% \beta_0 |\langle \psi(\vec{\theta})| \psi(\vec{\theta^0})\rangle  |^2
,
$$

for some positive $\beta_0$ that should ideally be greater than $\lambda_1-\lambda_0$. 

This new cost function can be interpreted as a constrained problem in which we minimize again $C_\text{VQE}(\vec{\theta}) = \langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle$ but, this time, we add the constraint that the state has to be orthogonal to the previously obtained $|\psi(\vec{\theta^0})\rangle$ with $\beta_0$ acting as a penalty term if orthogonality is not achieved. We are adding this constraint because the eigenstates of an observable (i.e. hermitian operator) corresponding to different eigenvalues are always mutually orthogonal —or can be made to be so in the event of degeneracy (i.e. repeated eigenvalues). It follows that, by enforcing orthogonality with the eigenstate corresponding to $\lambda_0$, we are _effectively_ optimizing over the subspace that corresponds to the rest of the eigenvalues $\{\lambda_1, \lambda_2,..., \lambda_{N-1}\}$; of which $\lambda_1$ is the lowest and, therefore, from the variational theorem, the optimal solution of the new problem.

This new problem can also be (more conveniently) interpreted as running VQE on the new observable:

$$
\hat{H_1} := \hat{H} + \beta_0 |\psi(\vec{\theta^0})\rangle \langle \psi(\vec{\theta^0})|
\quad \Rightarrow \quad 
C_1(\vec{\theta}) = \langle \psi(\vec{\theta}) | \hat{H_1} | \psi(\vec{\theta})\rangle,
$$

<!-- Derivation:
$$
\langle \psi(\vec{\theta}) | \hat{H_1} | \psi(\vec{\theta})\rangle = 
\langle \psi(\vec{\theta}) | \bigg( \hat{H} + 
\beta_0 |\psi(\vec{\theta^0})\rangle \langle \psi(\vec{\theta^0})| \bigg) | \psi(\vec{\theta})\rangle \\=
\langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle + 
\beta_0 \langle\psi(\vec{\theta}) |\psi(\vec{\theta^0})\rangle \langle \psi(\vec{\theta^0}) | \psi(\vec{\theta})\rangle \\=
\langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle + 
\beta_0 |\langle \psi(\vec{\theta})| \psi(\vec{\theta^0})\rangle  |^2 = 
C_1(\vec{\theta})
$$
-->

Assuming that the solution to the new problem is $|\psi(\vec{\theta^1})\rangle$, the expected value of $\hat{H}$ (not $\hat{H_1}$) should be $ \langle \psi(\vec{\theta^1}) | \hat{H} | \psi(\vec{\theta^1})\rangle = \lambda_1$.

In order to get the third eigenvalue $\lambda_2$, what you'd need to do is optimize the cost function:

$$
C_2(\vec{\theta}) := 
C_1(\vec{\theta}) + \beta_1 |\langle \psi(\vec{\theta})| \psi(\vec{\theta^1})\rangle  |^2 
% =
% \langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle + 
% \beta_0 |\langle \psi(\vec{\theta})| \psi(\vec{\theta^0})\rangle  |^2 +  
% \beta_1 |\langle \psi(\vec{\theta})| \psi(\vec{\theta^1})\rangle  |^2
,
$$

for some positive and big enough $\beta_1$. This time we'd be enforcing orthogonality of the solution state to both $|\psi(\vec{\theta^0})\rangle$ and $|\psi(\vec{\theta^1})\rangle$ by penalizing those states in the search space which do not meet such requirement; _effectively_ restricting the search space. Consequently, the optimal solution of the new problem should be the eigenstate corresponding to $\lambda_2$.

Similarly to the previous case, this new problem can also be conveniently interpreted as VQE with the observable:

$$
\hat{H_2} := \hat{H_1} + \beta_1 |\psi(\vec{\theta^1})\rangle \langle \psi(\vec{\theta^1})|
\quad \Rightarrow \quad 
C_2(\vec{\theta}) = \langle \psi(\vec{\theta}) | \hat{H_2} | \psi(\vec{\theta})\rangle.
% \hat{H_2} := 
% \hat{H} + \beta_0 |\psi(\vec{\theta^0})\rangle \langle \psi(\vec{\theta^0})| + 
% \beta_1 |\psi(\vec{\theta^1})\rangle \langle \psi(\vec{\theta^1})|.
$$

If the solution to this new problem is $|\psi(\vec{\theta^2})\rangle$, the expected value of $\hat{H}$ (not $\hat{H_2}$) should be $ \langle \psi(\vec{\theta^2}) | \hat{H} | \psi(\vec{\theta^2})\rangle = \lambda_2$.


Analogously, if you wanted to obtain the $k$-th eigenvalue $\lambda_{k-1}$, you'd minimize

$$
C_{k-1}(\vec{\theta}) := 
C_{k-2}(\vec{\theta}) + \beta_{k-2} |\langle \psi(\vec{\theta})| \psi(\vec{\theta^{k-2}})\rangle  |^2,
$$

Remember that we defined $\vec{\theta^j}$ such that $\langle \psi(\vec{\theta^j}) | \hat{H} | \psi(\vec{\theta^j})\rangle = \lambda_j$ $\forall j<k$. This problem would be equivalent to minimizing $C(\vec{\theta}) = \langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle$ but with the constraint that the state has to be orthogonal to $|\psi(\vec{\theta^j})\rangle \; \forall j \in \{0, \cdots, k-1\}$, therefore restricting the search space to the subspace corresponding to the eigenvalues $\{\lambda_{k-1},\cdots,\lambda_{N-1}\}$.

This problem is equivalent to a VQE with the observable

$$
\hat{H}_{k-1} := 
\hat{H}_{k-2} + \beta_{k-2} |\psi(\vec{\theta^{k-2}})\rangle \langle \psi(\vec{\theta^{k-2}})|
\quad \Rightarrow \quad 
C_{k-1}(\vec{\theta}) = \langle \psi(\vec{\theta}) | \hat{H}_{k-1} | \psi(\vec{\theta})\rangle,
$$

As you can see from the process, to get the $k$-th eigenvalue, you need the (approximate) eigenstates of the previous $k-1$ eigenvalues, so you'd need to run VQE a total of $k$ times. This fact may be easier to visualize by looking at the expanded form of the previous formulas, originally expressed in recursive form:

$$
C_{k-1}(\vec{\theta}) = 
\langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle + 
\sum_{j=0}^{k-2}\beta_j |\langle \psi(\vec{\theta})| \psi(\vec{\theta^j})\rangle  |^2, \\
\hat{H}_{k-1} := 
\hat{H} + \sum_{j=0}^{k-2}\beta_j |\psi(\vec{\theta^j})\rangle \langle \psi(\vec{\theta^j})|. 
$$

## Subspace Search VQE (SSVQE)

[SSVQE](https://arxiv.org/abs/1810.09434) is a variant of VQE that, like VQD, enables you to obtain the first $k$ eigenvalues of an observable $\hat{H}$ with eigenvalues $\{\lambda_0, \lambda_1,...,\lambda_{N-1}\}$, with $N\geq k$. We'll assume without loss of generality that $\lambda_0<\lambda_1<...<\lambda_{N-1}$.

This algorithm requires the usage of $k$ mutually orthogonal reference states $\{|\rho_j\rangle\}_{j=0}^{k-1}$, that is, $\langle \rho_j | \rho_l \rangle = \delta_{jl}$ for $j,l<k$. 

The (only) cost function of this algorithm is then:

$$ C(\vec{\theta}) := \sum_{j=0}^{k-1} w_j \langle \rho_j | U^{\dagger}(\vec{\theta})\hat{H} U(\vec{\theta})|\rho_j \rangle,$$

where $w_j$ is an arbitrary positive number such that if $j<l<k$ then $w_j>w_l$ and $U(\vec{\theta})$ is the user-defined variational form.

Like VQD, SSVQE hinges on the idea that eigenstates that correspond to different eigenvalues are mutually orthogonal. 

In particular, the inner product of $U(\vec{\theta})|\rho_j\rangle$ and $U(\vec{\theta})|\rho_l\rangle$ is given by $\langle \rho_j | U^{\dagger}(\vec{\theta})U(\vec{\theta})|\rho_l \rangle = \langle \rho_j | I |\rho_l \rangle = \langle \rho_j | \rho_l \rangle = \delta_{jl}$. The first equality holds because the ansatz $U(\vec{\theta})$ is a quantum operator and therefore unitary and the last one, because of the orthogonality of the reference states $|\rho_j\rangle$. This fact that orthogonality is preserved through unitary transformations is deeply related to the principle of conservation of information, as expressed in quantum information science. Under this view, non-unitary transformations represent processes where information is either lost or injected.

The question that arises now is: How can we ensure that those states are all eigenstates? That's where the weights $w_j$ become important. The idea is that, if the weights are different enough, the term with largest weight (i.e. $w_0$) will be given priority during optimization over the rest, so the resulting state $U(\vec{\theta})|\rho_0 \rangle$ will end up becoming the eigenstate corresponding to $\lambda_0$. Then, because of the mutual orthogonality of $\{U(\vec{\theta})|\rho_j\rangle\}_{j=0}^{k-1}$, the rest of the states will be orthogonal to it and, therefore, contained in the subspace corresponding to the eigenvalues $\{\lambda_1,...,\lambda_{N-1}\}$.

Applying the same argument to the rest of the terms, the next priority would then be the term with the weight $w_1$, so $U(\vec{\theta})|\rho_1 \rangle$ would be the eigenstate corresponding to $\lambda_1$ and the other terms would be contained in the eigenspace of $\{\lambda_2,...,\lambda_{N-1}\}$.

Reasoning inductively, we deduce that $U(\vec{\theta})|\rho_j \rangle$ will be an approximate eigenstate of $\lambda_j$ for $0\leq j < k $.

 




## Quantum Sampling Regression (QSR)

One of the main issues with VQE is that, in order to get get the parameters of the step $k$, you need the parameters from the step $k-1$ and so on, each one requiring a separate call to the quantum processor. This is a specially big problem when the access to the quantum devices is queued.

A way to circumvent this inconvenience is to use more classical resources to be able to do the full optimization process in one call. This is where [Quantum Sampling Regression](https://arxiv.org/pdf/2012.02338) comes into play.

The idea is that the cost function $C(\theta) := \langle \psi(\theta) | \hat{H} | \psi(\theta)\rangle$ can be written as a Fourier series the following way:

$$C(\theta) = \langle \psi(\theta) | \hat{H} | \psi(\theta)\rangle = a_0 + \sum_{k=1}^S[a_k\cos(k\theta)+ b_k\sin(k\theta)] $$

Depending on the periodicity and bandwidth of the original function, $S$ can be finite or infinite. We'll assume the latter to be the case.

The next step is to sample the cost function $C(\theta)$ several times to obtain the Fourier coefficients $\{a_0, a_k, b_k\}_{k=1}^S$. In particular, as we have $2S+1$ unknowns, we'll need to sample the cost function $2S+1$ times.

Then, if we sample the cost function for $2S+1$ paramter values $\{\theta_1,...,\theta_{2S+1}\}$, you'd obtain the following system:

$$\begin{pmatrix} 1 & \cos(\theta_1) & \sin(\theta_1) & \cos(2\theta_1) & ... & \sin(S\theta_1) \\
1 & \cos(\theta_2) & \sin(\theta_2) & \cos(2\theta_2) & \cdots & \sin(S\theta_2)\\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots\\ 
1 & \cos(\theta_{2S+1}) & \sin(\theta_{2S+1}) & \cos(2\theta_{2S+1}) & \cdots & \sin(S\theta_{2S+1})
\end{pmatrix} \begin{pmatrix} a_0 \\ a_1 \\ b_1 \\ a_2 \\ \vdots \\ b_S \end{pmatrix} = \begin{pmatrix} C(\theta_1) \\ C(\theta_2) \\ \vdots \\ C(\theta_{2S+1}) \end{pmatrix},$$

that we'll rewrite as 

$$Fa=c.$$

In practice, this system is generally not consistent, as the cost function values $c$ are not exact, so it is usually a good idea to normalize them by multiplying by $F^\dagger$ on the left, obtaining:

$$F^\dagger Fa = F^\dagger c.$$

This new system is always consistent and its solution is a least-squares solution to the original problem.

If instead of only one parameter we have $k$, each parameter $\theta^i$ will have its own $S_i$ for $i\in \{1,...,k\}$, so the total number of samples needed would be:

$$T=\prod_{i=1}^k(2S_i+1)\leq \prod_{i=1}^k(2S_{max}+1) = (2S_{max}+1)^n,$$

where $S_{max}=\max_i(S_i)$.

Furthermore, playing around with $S_{max}$ as a tunable parameter (i.e. instead to inferring it) opens up new possibilities like:
- _Oversampling_: to improve accuracy.
- _Undersampling_: to boost performance by reducing runtime overhead, or eliminating local minima.

Refinement and combinations with other techniques are also possible.

In [1]:
import qiskit.tools.jupyter
%qiskit_copyright